# Intelligence Artificielle Avancée
## TP5 : Introduction au Deep Learning

Au cas où, il nous faut d'abord vérifier la version du Keras et Tensorflow:

In [ ]:
import tensorflow as tf
from tensorflow.python import keras
print("Keras:", keras.__version__)
print("Tensorflow:", tf.__version__)


**Installer Pydot et Graphiz, si ne sont pas déjà installés:**

Puis vous devez relancer le noyau : trouver 'restart kernel' dans les menus.

## De la documentation


### Deep Learning avec Keras
- Doc keras :https://keras.io/
- Le model Sequential (premier pas) : https://keras.io/getting-started/sequential-model-guide/#getting-started-with-the-keras-sequential-model
- Un framework plus riche : https://keras.io/getting-started/functional-api-guide/

### Autres Toolkits et packages

- Lasagne : Langage *de haut niveau* comme keras


NB : Lasagne et Keras utilisent indifférement un backend parmi Theano (Univ. Montreal) et Tensorflow (Google)

- Theano :  le package de l'Univ. de Montréal

- Tensorflow :  le package Google

- Caffe : très spécialisé images

- Pytorch : la plateforme de Facebook

- ...


## Récupération d'un dataset existant
On va travailler avec un jeux de données historique que vous conaissez déjà : MNIST. Ce sont des chiffres manuscrits, donc des images en niveaux de gris, en résolution 28x28 cette fois (cela peut prendre un peu de temps - et générer un FutureWarning, ce qui n'est pas grave) :

In [3]:
from keras.datasets import mnist
from keras.utils import np_utils
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

print("dimension des données : ", X_train.shape, y_train.shape)
print("une données : ")
print(X_test[42])

print("Une données sous format plus lisible :")
plt.imshow(X_train[42], cmap=plt.get_cmap('gray'))
plt.show()

### Prétraitement
Les données ne sont pas dans le format nécessaire à Keras : il vaut vectoriser chaque image. On va aussi les normaliser, pour n'avoir que des valeurs comprisent entre 0 et 1 (améliore la vitesse de convergence)

In [5]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

import numpy as np
X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))

In [ ]:
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print("Classe de la data num 42 :", y_train[42])

#### One-hot-encoding
Les réseaux de neurones ont besoin d'un vecteur à la place de *y* : on utilise un codage où la taille de ce vecteur est le nombre de classe, toutes les valeurs du vecteurs sont égales à 0, sauf celle qui correspond à la bonne classe qui elle vaut 1.

In [7]:
# On transforme les sorties (numéros de classe) en des vecteurs de type one-hot-code
nb_classes = 10

Y_train = y_train
Y_test = y_test
y_train =  np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)


In [ ]:
print(y_train.shape)
print("Classe de la data num 42 : ", y_train[42])

## Apprendre un réseau de neurones pour la classification de MNIST
D'abord il faut créer le model.
On ajoute les couches une à une. Ici un modèle qui :
- prend en entrée un vecteur de dimension 784 (une image Mnist 28x28 vectorisée)
- transforme l'entrée en un vecteur de dimension 64 avec une couche totalement connectée (Dense), avec une fonction d'une activation de type Rectified Linear Unit 
- Transforme la sortie de la couche précédente (de dimension 64) en un vecteur de dimension 10 avec une autre couche dense 
- Transforme le vecteur de dimenbsion 10 en un autre vecteur de dimension 10 à l'aide d'une couche dense avec la fonction d'activation softmax

In [9]:
# Import from Keras
from keras.layers import Input, Dense
from keras.models import Model, Sequential


On crée note réseau de neurones (cela génère un warning désagréable mais qui ne pause pas de problème finalement) :

In [10]:
model = Sequential()
model.add(Dense(64, input_dim=784, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='softmax'))

Puis il faut "compiler" le modèle, en précisant :
- le critère d'optimisation : le *loss*
- la routine d'optimisation (ie l'utilisation du gradient) : l'*optimizer*
- les métriques additionnelles au *loss* (ici l'*accuracy*, le taux de bonne classification) que l'on va calculer à chaque fois

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['acc'])

### Affichage de la structure du modèle

Expliquee les différentes éléments de chacune des lignes affichées par la commande suivante.

In [ ]:
model.summary()

Ou une manière plus graphique (nécessite l'installation de pydot et surtout de graphviz, ce qui n'est pas forcément facile) :

In [ ]:
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

### On peut maintenant apprendre le modèle en précisant 
- la base d'apprentissage (les deux premiers paramètres)
- le nombre d'itérations d'apprentissage (*epochs*)
- la taille des minibatchs (*batch_size*)
- un ensemble de validation (soit on utilise comme ici un pourcentage des données d'entrée *X_train, y_train*, soit d'autres ensemble de données via *validation_data=(X_test, y_test)*)
- le niveau de verbosité de l'affichage

*(là encore, on a un WARNING, mais il n'empêche pas le code de fonctionner)*

In [ ]:
h = model.fit(X_train, y_train,
              epochs=3,
              batch_size=16,
              verbose =1,
              validation_split=0.33)

### Evaluation
On peut alors évaluer le modèle sur l'ensemble de test

In [15]:
score = model.evaluate(X_test, y_test, verbose=2, batch_size=16)

print (score)

625/625 - 0s - loss: 2.1733 - acc: 0.2016
[2.1733250617980957, 0.20160000026226044]


### Historique de processus
Les info sur le processus d'apprentissage sont stockées dans *h*.

Comme on va en avoir besoin plusieurs fois, on écrit une fonction qui prend un historique d'apprentissage et affiche les courbes : une pour la fonction de perte (loss) et l'autre pour le taux de réussite (accuracy) :

In [16]:
def affiche_evolution_apprentissage(history):
    #affiche history.history.keys()
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('accuracy du modèle')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['données apprentissage', 'données test'], loc='upper left')
    plt.show()
    # résumé de l'historique pour loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('loss du modèle')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['apprentissage', 'test'], loc='upper left')
    plt.show()

In [ ]:
affiche_evolution_apprentissage(h)

#### En utilisant les réseaux de neurones convolutionnels, proposer un deuxième modèle sur la même donnée (MNIST), le modèle proposé sera constité de deux couches (Conv2D de taille 3), après chaque couche de convoltion une couche d'échantillonnage  (MaxPooling2D de taille 2) est fortement recommandée.   

In [ ]:
#A vous

#### Evaluer les nouveau model construit (CNN) et le comparer avec le modèle dense (MLP)

In [ ]:
#a vous

#### Evaluer les hyperparamètres droupout et batch-size sur la perfermance du modèle

In [2]:
#A vous

### Callback
Permet de programmer la sauvegarde des modèles à chaque itération, l'adaptation du pas d'apprentissage (learning rate), une procédure de early stopping, etc.

Voir <https://keras.io/callbacks/> pour les détails

In [ ]:
# Early Stopping
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=10, verbose=1, mode='auto')

# Adaptation du pas d'apprentissage
from keras.callbacks import ReduceLROnPlateau
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', 
                       min_delta=0.0001, cooldown=0, min_lr=0)

h_es_lr = model.fit(X_train, y_train,
                    epochs=2,
                    batch_size=16,
                    verbose =1,
                    validation_split=0.33,
                    callbacks=[es, lr])

## Sauver et récupérer des models

In [ ]:
from keras.models import load_model

# Eventuellement, installation d'un module nécessaire :
#!pip3 install h5py

score = model.evaluate(X_test, y_test, batch_size=16)
print ("Initialement : ", score)

# Sauver le model 
model.save('mon_modele.h5')  # crée un fichier HDF5del model  

# supprime le modèle
del model

# Récupérer le modèle   
model = load_model('mon_modele.h5')

score = model.evaluate(X_test, y_test, batch_size=16)
print ("Après suppression-récupération : ", score)
print(model)

## Utiliser Keras dans un code SciKit-learn (cross-validation / Grid search)

Dans Scikit-Learn, il y a une classe *KerasClassifier* qu'il peut-être utile de savoir utiliser :

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model_Mnist(optimizer='rmsprop', input_datadim = 784, init='glorot_uniform', nb_hid1= 20, do_rate= 0.5 ):
    # fonction créant un modèle pour MNIST
    """
    #Jusqu'à présent :
    m = Sequential()
    m.add(Dense(nb_hid1, input_dim=input_datadim, activation='relu'))
    m.add(Dropout(do_rate))
    m.add(Dense(64, activation='relu'))
    m.add(Dropout(do_rate))
    m.add(Dense(10, activation='softmax'))
    """
    # De façon équivalente :
    entree= Input(shape=(784,))
    cachee_1= Dense(64, activation='relu')(entree)
    cachee_2 = Dense(64, activation='relu')(cachee_1)
    sortie = Dense(10, activation="softmax")(cachee_2)
    m = Model(entree, sortie)
    
    m.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
    m.summary()
    return m

model = KerasClassifier(build_fn=create_model_Mnist)

# valeurs des différents paramètres
optimizers = ['adam', 'rmsprop']
init = ['glorot_uniform']
V_nb_hid1 = [100]
DO_rate=[0, 0.5]

epochs = 2

param_grid = dict(optimizer=optimizers, init=init,  nb_hid1=V_nb_hid1, do_rate= DO_rate)

print(param_grid)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train, epochs = epochs, verbose=2)

# Résumé des résultats
print("-----------------------------")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

print("-----------------------------")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

### Récupération des vecteurs d'activation d'une couche
Une fois un réseau appris, on peut avoir besoin des vecteurs de sortie d'une couche cachée. Avec Keras, il suffit de créer un nouveau modèle qui ne contient que le début du réseau jusqu'à la couche dont on veut accéder aux sorties.

In [ ]:
#réseau complet
entree= Input(shape=(784,))
cachee_1= Dense(64, activation='relu')(entree)
cachee_2 = Dense(64, activation='relu')(cachee_1)
sortie = Dense(10, activation="softmax")(cachee_2)
m = Model(entree, sortie)

#apprentissage
m.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
m.fit(X_train, y_train, epochs=2, batch_size=16, verbose =1, validation_split=0.33)

In [23]:
#réseau partiel
m2 = Model(entree,cachee_1)

#On aurait aussi pu faire :
# m2 = Sequential()
# m2.add(Dense(64,input_dim=784, activation='relu', weights=m.layers[0].get_weights()))

#récupération d'une matrice d'activation :
#chaque ligne est le vecteur de sortie de la dernière couche de m2 pour la donnée correspondante
activite = m2.predict(X_test)
print(activite.shape)

(10000, 64)


# A faire

## Apprentissage d'un réseau de neurones sur les données iris


Apprendre un réseau de neuronnes réalisant la classification 3 classes sur les données iris.


**Question**. En utilisant Pandas, lire les données iris à partir de la bibliothèque Sklearn et compléter les instructions ci-dessous:

In [81]:
from sklearn.datasets import load_iris
import pandas as pd


# Lire les données iris
iris = load_iris()


# Charger des données dans un DataFrame

#..........Ecrire votre instruction ici............


# Convertir le type de données en float

#..........Ecrire votre instruction ici............


# ajoutez "target" dans le DataFrame et nommez-le "label"

#..........Ecrire votre instruction ici............

# Utilisez plutôt une étiquette de chaîne
df['label'] = df.label.replace(dict(enumerate(iris.target_names)))


**Question.** Afficher le contenu du DataFrame

**Question.** Transformer les labels de classes en des vecteurs de type one-hot-code et supprimer l'ancien label

**Question.** Ré-afficher le contenu du nouveau DataFrame.

**Question.** Donner une petite explication sur la différence entre l'ancien DatFrame et celui-ci.

**Question.** Extraire les données x (samples) et y (classes) à partir des données iris, en les convertant en Numpy pour faire la classification (En utilisant toujours Pandas)

**Question.** Découper les données en ensembles de training et test (20 % pour le test).

**Question.** Construire un modèle du réseau de neurones séquentiel: 3 couches cachées (64, 128, 64) et les fonctions d'activation sont relu() et softmax().

**Question.** Visualiser l''arborescence du modèle construit. Commenter!

**Question.** Compiler et entraîner votre modèle avec le nombre d'epochs=15, l'ensemble de validation=25% et  batch size=40. Vous pouvez utiliser Adam comme optimiseur. A vous de choisir une fonction et métrique de perte adéaquates  pour ce type de problème. Réflichissez!

**Question.** Tracez les courbes d'apprentissage (train_loss et val_loss)

**Question.** Interpréter le résultat obtenu. Que constatez-vous ? 

**Question.** Après avoir construit le modèle, utiliser la commande  EarlyStopping() sans paramètres sous Keras pour éviter le problème d'Overfitting. Re-entraîner le modèle.

**Question.**  Interpréter le résulat

**Question.** Re-tracez les courbes d'apprentissage (train_loss et val_loss)

**Question.**  Interpréter le résulat des courbes d'apprentissage obtenus

**Question.** Après avoir essayer EarlyStopping() sans paramètres, personaliser maintenant cette commande en fixant les les paramètres min_delta=$1e-3$, patience=8 et mode='max'.


**Question.** Expliquer les paramètres suivants: min_delta, patience et verbose.

**Question.** Entraîner votre modèle en tenant en compte l'instruction d'EarlySopping personalisé.

**Question.** Interpréter le résultat

**Question.** Tracez les courbes d'apprentissage (train_loss et val_loss)

**Question.** Que constatez-vous ?

## Régression avec EarlyStopping

Après avoir apprendre un modèle de classification, maintenant on s'intéresse à un problème de régression. 

En utilisant Pandas, nous allons charger le fichier CSV des données auto à partir de cette URL: "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv". Dans la même instruction, nous vérifiions s'il y a des valeurs manquantes comme 'NA' ou '?' et les stocker dans une variable nommée na_values.

In [111]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Gérer les valeurs manquantes
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

**Question.** Convertir les données en Numpy pour faire la régression (données x et y)

**Question.** Découper les données en ensembles de training et test avec la taille de test=25%

**Question.** Constuire un modèle de réseau de neurones séquentiel contenant 2 couches cachées (25, 10). Attention: Il faut choisir la métrique de perte la plus adéquate! Vous pouvez utiliser ADAM comme l'exercice précédent.

**Question.** Entraîner maintenant le modèle avec un nombre d'epochs=300. Vous ajouterez la méthode EarlyStopping() avec min delta=$1e-3$, patience=5 et mode='auto'. Afficher et commenter les résultats obtenus.